In [2]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import iisignature
import itertools
from tqdm import tqdm

import warnings 
warnings.filterwarnings('ignore')


the transform function from augment do several good transformation for the data set, it adds a bespoint, it normalizes the data, it add a time feature ( i.e. use timestep = {1,2,3..} to make time =[0.00, 0.33, 0.66, ...]), it finally lags the features ( move L step forward). 

Then the transform method from the class fetaure extraction compute the signautre, first it initialize a depth for the signature, then it encapsulates the data set into levels ( levels of the order book , each level has 4 features bid ask prices and bid asd volums) and for each subset that correspond to a level we add the midprice columns ( the last columns). Se for each level we have a dataset of 5 columns. Then we apply the augment method defined previously and we finally compute the signature using the signature library. The size of X_out is then the number of levels. 

In [8]:
class Augment(BaseEstimator, TransformerMixin):

    def __init__(self, normalize=False, add_time=True, lead_lag=False, basepoint=True, t0=0.0, t1=1.0, lags=[1]):
        self.normalize = normalize
        self.add_time = add_time
        self.lead_lag = lead_lag
        self.basepoint = basepoint
        self.t0, self.t1 = t0, t1
        self.lags = lags

    def fit(self, X):
        return self
  
    def transform(self, X):
        if self.normalize:
            X /= X.max(axis=1)[:, None, :]
        if self.lead_lag:
            X_list = [X] 
            for lag in self.lags:
                X_shifted = np.zeros_like(X)
                X_shifted[:, lag:, :] = X[:, :-lag, :]                
                X_shifted[:, :lag, :] = np.expand_dims(X[:, 0, :], axis=1)
                X_list.append(X_shifted)
            X = np.concatenate(X_list, axis=-1)
        if self.add_time:
            time = np.linspace(self.t0, self.t1, X.shape[1])
            X = np.concatenate((np.tile(time[None, :, None], [X.shape[0], 1, 1]), X), axis=-1)
        if self.basepoint:
            X = np.concatenate((np.zeros_like(X[:, :1]), X), axis=1)
        return X
    
class FeatureExtraction(Augment):

    def __init__(self, 
                 depth=2, 
                 t0=0.0, 
                 t1=1.0, 
                 lags=[1], 
                 levels=1,
                 level_by_level=True, 
                 normalize=False, 
                 add_time=True, 
                 lead_lag=False, 
                 basepoint=False
                 ):
        
        Augment.__init__(self, normalize, add_time, lead_lag, basepoint, t0, t1, lags)
        self.depth = depth
        self.levels = levels
        self.level_by_level = level_by_level

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_sig_list = []
        indices = np.arange(self.levels) if self.level_by_level else [None]
        for j in indices: 
            if j is not None:
                start_idx = 4 * j # for each level we capture 4 columns , levels are the levels of the order book)
                end_idx = 4 * (j + 1)
            else:
                start_idx = 0
                end_idx = 4 * self.levels
            X_window = X[:, :, start_idx:end_idx]
            X_midprice = X[:, :, -1:]
            X_window = np.concatenate([X_window, X_midprice], axis=2)
            X_augment = Augment.transform(self, X_window)
            X_sig = iisignature.sig(X_augment, self.depth)
            X_sig_list.append(X_sig)
        X_out = np.concatenate(X_sig_list, axis=1)
        return X_out


In [5]:
nrows = 500000
data = pd.read_csv('/Users/arthur/Documents/STUDY/Imperial/rough paths /salvi notebook/data.csv.gz', compression='gzip', nrows=nrows)

for col in data.columns:
        print(col)

askRate0
askRate1
askRate2
askRate3
askRate4
askRate5
askRate6
askRate7
askRate8
askRate9
askRate10
askRate11
askRate12
askRate13
askRate14
askSize0
askSize1
askSize2
askSize3
askSize4
askSize5
askSize6
askSize7
askSize8
askSize9
askSize10
askSize11
askSize12
askSize13
askSize14
bidRate0
bidRate1
bidRate2
bidRate3
bidRate4
bidRate5
bidRate6
bidRate7
bidRate8
bidRate9
bidRate10
bidRate11
bidRate12
bidRate13
bidRate14
bidSize0
bidSize1
bidSize2
bidSize3
bidSize4
bidSize5
bidSize6
bidSize7
bidSize8
bidSize9
bidSize10
bidSize11
bidSize12
bidSize13
bidSize14
y


In [9]:
nrows = 500000

data = pd.read_csv('/Users/arthur/Documents/STUDY/Imperial/rough paths /salvi notebook/data.csv.gz', compression='gzip', nrows=nrows)

# set the NaN sizes sizes to zero
askSize_columns = [f'askSize{i}' for i in range(15)] 
bidSize_columns = [f'bidSize{i}' for i in range(15)]
size_columns = askSize_columns + bidSize_columns
data[size_columns] = data[size_columns].fillna(0.0)
# set the NaN rates to last observed value and fill the remaining ones backward
askRate_columns = [f'askRate{i}' for i in range(15)] 
bidRate_columns = [f'bidRate{i}' for i in range(15)]
rate_columns = askRate_columns + bidRate_columns
data[rate_columns] = data[rate_columns].ffill().bfill()
data['midprice'] = 0.5 * (data['askRate0'] + data['bidRate0']) 

# reorder columns
selected_columns = [[f'askRate{i}', f'bidRate{i}', f'askSize{i}', f'bidSize{i}'] for i in range(15)]
selected_columns = list(itertools.chain.from_iterable(selected_columns)) + ['midprice']
data = data[selected_columns]
for col in data.columns:
        print(col)

askRate0
bidRate0
askSize0
bidSize0
askRate1
bidRate1
askSize1
bidSize1
askRate2
bidRate2
askSize2
bidSize2
askRate3
bidRate3
askSize3
bidSize3
askRate4
bidRate4
askSize4
bidSize4
askRate5
bidRate5
askSize5
bidSize5
askRate6
bidRate6
askSize6
bidSize6
askRate7
bidRate7
askSize7
bidSize7
askRate8
bidRate8
askSize8
bidSize8
askRate9
bidRate9
askSize9
bidSize9
askRate10
bidRate10
askSize10
bidSize10
askRate11
bidRate11
askSize11
bidSize11
askRate12
bidRate12
askSize12
bidSize12
askRate13
bidRate13
askSize13
bidSize13
askRate14
bidRate14
askSize14
bidSize14
midprice


In [10]:

# Lag values
lags = [57, 65, 75, 80, 87, 100]
# Compute target for each lag
for lag in lags:
    target_data = pd.DataFrame({
        'timestep': data.index[:-lag],
        f'target_lag_{lag}': data['midprice'].shift(-lag)[:-lag].values - data['midprice'][:-lag].values
    })
    
    # Save the target data to CSV
    filename = f'lag_{lag}_target.csv'
    target_data.to_csv(filename, index=False)

target_data.head()

,timestep,target_lag_100
0,0,-0.50
1,1,-0.50
2,2,-0.50
3,3,-0.25
4,4,-0.25


the window transform create a matrix of size (n, d, M ) with n the number of time step, d the number of features and M the size of the widow. the temp object contains all the different window sliding into the time step ( if n =10, M=5 : 6 windows) and each window has d features. The X_windows still have n rows but the the M-1 (  size of the windows) rows are empty because there is no espace to have a window, then each row is equal to a window containing the M previous values. The leave out start and leave out end cut the end and the beginning of the X_window to have only the core information, it is set to 500 as the max window size is 200. 

Next for each depth ( S(k)), 
    for each level (level of order book): 
        we create an object of feature extraction that contains the method signature, chunksize is the number of parallel computation
for the dimension we will have data of 6 features : 4 features per level ( bid/ask/price/volume)
the midprice and the basepoint , and the signature are added one to eachother : 90 = 6*15; 630 = 6*6*15+90 , 3870 = 6*6*6*15+630,

In [18]:
leave_out_start = 500
leave_out_end = 500

M, N = data.shape

# extract signature features
for window in tqdm([ 50, 87]):
    # segment data
    temp = np.lib.stride_tricks.sliding_window_view(data, window_shape=window, axis=0).transpose(0, 2, 1)
    X_windows = np.full((M, window, N), np.nan)
    X_windows[window - 1:, ...] = temp
    X_windows = X_windows[leave_out_start:-leave_out_end, ...]
    for depth in [1, 2,3]:
        for levels in [ 15]:
            feature_extraction = FeatureExtraction(depth=depth, levels=levels)
            chunk_size = 1000  
            num_chunks = (X_windows.shape[0] + chunk_size - 1) // chunk_size 
            sub_arrays = np.array_split(X_windows, num_chunks, axis=0)
            all_transformed_chunks = []
            for chunk in sub_arrays:
                X_transformed_chunk = feature_extraction.fit_transform(chunk)
                all_transformed_chunks.append(X_transformed_chunk)
            X = np.concatenate(all_transformed_chunks, axis=0)
            assert not np.isnan(X).any(), "NaNs values found in signature features"
            print(X.shape)
            print(f"Processed window {window}, level {levels}, depth {depth}, shape: {X.shape}")

            # Convert to Pandas DataFrame
            df = pd.DataFrame(X)

            # Define filename with format: dataset_window-{window}_level-{level}_depth-{depth}.csv
            filename = f"dataset_window-{window}_level-{levels}_depth-{depth}.csv"

            # Save dataset as CSV
            df.to_csv(filename, index=False)
            print(f"Saved: {filename}")


  0%|          | 0/2 [00:00<?, ?it/s]

(499000, 90)
Processed window 50, level 15, depth 1, shape: (499000, 90)
Saved: dataset_window-50_level-15_depth-1.csv
(499000, 630)
Processed window 50, level 15, depth 2, shape: (499000, 630)
Saved: dataset_window-50_level-15_depth-2.csv
(499000, 3870)
Processed window 50, level 15, depth 3, shape: (499000, 3870)


 50%|█████     | 1/2 [56:52<56:52, 3412.66s/it]

Saved: dataset_window-50_level-15_depth-3.csv


: 

this will rename the columns of the signatures so that we can understand it easily. 

## dont forget to suffle the data set for the entry 


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools

leave_out_start = 500
leave_out_end = 500
nrows = 500000

M, N = data.shape  # Get dataset shape
window_sizes = [50, 87]  # Define window sizes
levels = 15  # Number of order book levels
depths = [1, 2, 3]  # Signature depths
num_features_per_level = 6  # **6 features per level (5 original + basepoint)**

# Initialize storage for the final dataset
final_dataset = []

for window in tqdm(window_sizes):
    # Step 1: Segment the data into rolling windows
    temp = np.lib.stride_tricks.sliding_window_view(data, window_shape=window, axis=0).transpose(0, 2, 1)
    X_windows = np.full((M, window, N), np.nan)
    X_windows[window - 1:, ...] = temp
    X_windows = X_windows[leave_out_start:-leave_out_end, ...]  # Trim unwanted rows

    # Extract last timestamp for each window (most recent time in window)
    time_column = data.index[leave_out_start + window - 1:-leave_out_end].values

    # Initialize empty list to store all levels' signature features
    signature_features = []
    column_names = ["time"]  # First column is time

    for depth in depths:
        feature_extraction = FeatureExtraction(depth=depth, levels=levels)
        
        # Chunking mechanism to avoid memory overflow
        chunk_size = 1000  
        num_chunks = (X_windows.shape[0] + chunk_size - 1) // chunk_size
        sub_arrays = np.array_split(X_windows, num_chunks, axis=0)

        all_transformed_chunks = []

        # Process each chunk and extract features
        for chunk in sub_arrays:
            X_transformed_chunk = feature_extraction.fit_transform(chunk)
            all_transformed_chunks.append(X_transformed_chunk)

        # Concatenate transformed data across chunks
        X = np.concatenate(all_transformed_chunks, axis=0)

        # Ensure no NaN values
        assert not np.isnan(X).any(), "NaNs values found in signature features"
        print(f"Processed window {window}, level {levels}, depth {depth}, shape: {X.shape}")

        # Append signature features for each depth
        signature_features.append(X)

        # Step 4: Generate column names dynamically
        for level in range(1, levels + 1):
            if depth == 1:  # Depth 1 (Vector)
                for f1 in range(num_features_per_level):
                    column_names.append(f"level_{level}_sig_depth_1_feature_{f1}")

            elif depth == 2:  # Depth 2 (Matrix)
                for f1, f2 in itertools.product(range(num_features_per_level), repeat=2):
                    column_names.append(f"level_{level}_sig_depth_2_feature_{f1}_feature_{f2}")

            elif depth == 3:  # Depth 3 (Tensor)
                for f1, f2, f3 in itertools.product(range(num_features_per_level), repeat=3):
                    column_names.append(f"level_{level}_sig_depth_3_feature_{f1}_feature_{f2}_feature_{f3}")

    # Concatenate all computed signatures for Depth 1, 2, 3
    X_final = np.concatenate(signature_features, axis=1)
   
    data = pd.read_csv('/Users/arthur/Documents/STUDY/Imperial/rough paths /salvi notebook/data.csv.gz', compression='gzip', nrows=nrows)

    # Extract y target values (corresponding to the last row in each window)
    y_values = data.iloc[leave_out_start + window - 1:-leave_out_end, -1].values  # Assuming y is the last column

    # Create DataFrame
    df = pd.DataFrame(X_final, columns=column_names[1:])  # Skip "time" for now

    # Add time and target column to DataFrame
    df.insert(0, "time", time_column)
    df["y"] = y_values

    # Save dataset
    filename = f"final_dataset_window-{window}_levels-{levels}_depths-{len(depths)}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved: {filename}")

    # Append to final dataset list
    final_dataset.append(df)

# Concatenate final dataset for all window sizes if needed
final_df = pd.concat(final_dataset, axis=0)
final_df.to_csv("complete_signature_dataset.csv", index=False)
print("Final dataset saved as complete_signature_dataset.csv")
